In [1]:
import pandas as pd
import Levenshtein
import itertools
from fuzzywuzzy import fuzz, process
import ast
from selenium import webdriver
import time
import webbrowser

In [2]:
recipe_df = pd.read_csv('../data/processed/ingredients_df.csv')

pd.set_option('max_columns', None)

recipe_df.head()

,recipe_image_url,recipe_name,recipe_url,ingredients_list,time_preparation(min),gluten_free,egg_free,sucrose_fructose_free,low_sodium,lactose_free,main_category,num_ingredients,variable,ingredient
0,https://content-recetas.lecturas.com/medio/201...,Ensalada niçoise,https://www.lecturas.com/recetas/ensalada-nico...,"['patatas pequeñas rojas', 'judías verdes', 't...",60.0,0,0,0,0,0,principal,12,0,patatas pequeñas rojas
1,https://content-recetas.lecturas.com/medio/201...,Ensalada del périgord,https://www.lecturas.com/recetas/ensalada-peri...,"['foie micuit', 'beicon gruesas', 'champiñones...",25.0,0,0,0,0,0,principal,9,0,foie micuit
2,https://content-recetas.lecturas.com/medio/202...,Xató con salsa romesco,https://www.lecturas.com/recetas/xato-salsa-ro...,"['bacalao desalado', 'cebolleta', 'escarolas',...",35.0,0,0,0,0,0,principal,16,0,bacalao desalado
3,https://content-recetas.lecturas.com/medio/201...,Ensalada de ceviche de langostinos y cebolla,https://www.lecturas.com/recetas/ensalada-cevi...,"['langostinos', 'limas', 'tomates', 'cebolla m...",25.0,0,0,0,0,0,principal,12,0,langostinos
4,https://content-recetas.lecturas.com/medio/201...,"Ensalada de pollo, remolacha y manzana",https://www.lecturas.com/recetas/ensalada-poll...,"['manzanas ácida', 'col lombarda', 'granada', ...",40.0,0,0,0,0,0,principal,16,0,pechugas de pollo


In [3]:
input_list = ['huevos', 'pechiga de pollo', 'lentejas', 'pimiento', 'arroz', 'leche', 'mantequilla',
              'harina', 'sal', 'azúcar', 'aceite', 'zanahorias', 'queso', 'bacon', 'lechuga', 'calabacín',
              'patatas', 'pollo', 'atún', 'garbanzos', 'judías verdes', 'jamón', 'jamón cocido', 'pan', 
              'pasta']

In [4]:
def fuzzy_columns(column, input_string):
    ratio = fuzz.WRatio(column, input_string)
    return ratio

In [5]:
for ingred in input_list:
    recipe_df[ingred] = recipe_df.apply(lambda x: fuzzy_columns(x["ingredient"], ingred), axis=1)

In [6]:
recipe_df

,recipe_image_url,recipe_name,recipe_url,ingredients_list,time_preparation(min),gluten_free,egg_free,sucrose_fructose_free,low_sodium,lactose_free,main_category,num_ingredients,variable,ingredient,huevos,pechiga de pollo,lentejas,pimiento,arroz,leche,mantequilla,harina,sal,azúcar,aceite,zanahorias,queso,bacon,lechuga,calabacín,patatas,pollo,atún,garbanzos,judías verdes,jamón,jamón cocido,pan,pasta
0,https://content-recetas.lecturas.com/medio/201...,Ensalada niçoise,https://www.lecturas.com/recetas/ensalada-nico...,"['patatas pequeñas rojas', 'judías verdes', 't...",60.0,0,0,0,0,0,principal,12,0,patatas pequeñas rojas,45,32,34,22,36,36,40,30,30,36,30,36,72,18,39,34,90,18,60,40,45,45,24,60,72
1,https://content-recetas.lecturas.com/medio/201...,Ensalada del périgord,https://www.lecturas.com/recetas/ensalada-peri...,"['foie micuit', 'beicon gruesas', 'champiñones...",25.0,0,0,0,0,0,principal,9,0,foie micuit,15,28,21,42,12,25,27,15,0,18,45,19,18,18,39,11,13,18,30,10,9,22,34,0,18
2,https://content-recetas.lecturas.com/medio/202...,Xató con salsa romesco,https://www.lecturas.com/recetas/xato-salsa-ro...,"['bacalao desalado', 'cebolleta', 'escarolas',...",35.0,0,0,0,0,0,principal,16,0,bacalao desalado,30,44,34,22,36,19,30,30,90,54,30,36,36,54,26,45,39,36,30,40,36,22,37,30,36
3,https://content-recetas.lecturas.com/medio/201...,Ensalada de ceviche de langostinos y cebolla,https://www.lecturas.com/recetas/ensalada-cevi...,"['langostinos', 'limas', 'tomates', 'cebolla m...",25.0,0,0,0,0,0,principal,12,0,langostinos,30,22,42,32,36,18,36,35,30,18,30,48,36,38,26,32,33,25,60,40,26,45,45,60,38
4,https://content-recetas.lecturas.com/medio/201...,"Ensalada de pollo, remolacha y manzana",https://www.lecturas.com/recetas/ensalada-poll...,"['manzanas ácida', 'col lombarda', 'granada', ...",40.0,0,0,0,0,0,principal,16,0,pechugas de pollo,45,91,34,24,34,54,32,30,30,18,30,26,36,18,77,24,26,90,30,30,41,22,32,30,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174669,https://t1.rg.ltmcdn.com/es/images/4/4/6/img_p...,"Pavo relleno con castaÑas, higados de pollo y ...",https://www.recetasgratis.net/receta-de-pavo-r...,"['ingredientes ', 'a kilos', 'o limones partid...",NaN,0,0,0,0,0,principal,39,38,la pulpa de una manzana,15,34,34,34,36,18,38,45,30,36,15,43,18,36,39,43,39,36,60,40,38,45,24,60,36
174670,https://t2.rg.ltmcdn.com/es/images/9/1/9/img_l...,Langostinos arco iris,https://www.recetasgratis.net/receta-de-langos...,"['3langostinos medianos, tipo marfil,', 'oliva...",NaN,0,0,0,0,0,principal,39,38,azúcar.,0,19,18,0,40,20,25,36,30,100,36,40,0,40,33,36,33,0,30,36,24,22,36,30,40
174671,https://t2.rg.ltmcdn.com/es/images/5/4/6/img_p...,Pavo relleno con arroz salvaje,https://www.recetasgratis.net/receta-de-pavo-r...,"['a kilos', 'limones', 'cebolla', 'vinagre', '...",480.0,0,0,0,0,0,principal,42,39,pollo,18,90,15,31,20,20,36,0,30,0,0,18,20,20,17,18,17,100,0,18,0,0,36,30,20
174672,https://t2.rg.ltmcdn.com/es/images/5/4/6/img_p...,Pavo relleno con arroz salvaje,https://www.recetasgratis.net/receta-de-pavo-r...,"['a kilos', 'limones', 'cebolla', 'vinagre', '...",480.0,0,0,0,0,0,principal,42,40,azúcar,0,19,18,0,40,20,25,36,30,100,36,40,0,40,33,36,33,0,30,36,24,22,36,30,40


In [7]:
input_list

['huevos',
 'pechiga de pollo',
 'lentejas',
 'pimiento',
 'arroz',
 'leche',
 'mantequilla',
 'harina',
 'sal',
 'azúcar',
 'aceite',
 'zanahorias',
 'queso',
 'bacon',
 'lechuga',
 'calabacín',
 'patatas',
 'pollo',
 'atún',
 'garbanzos',
 'judías verdes',
 'jamón',
 'jamón cocido',
 'pan',
 'pasta']

In [43]:
filter_huevos = recipe_df['huevos'] > 86

recipe_df[filter_huevos]["ingredient"].value_counts().index.tolist()


['huevo',
 'huevos',
 'huevos cocidos',
 'huevos duros',
 'huevo ',
 'huevos de codorniz',
 'huevos.',
 'huevos grandes',
 'huevos batidos',
 'huevos ',
 'huevos enteros',
 'huevos medianos',
 'huevo.',
 '1huevos',
 'huevos l',
 'huevos duros picados',
 'tres huevos',
 'huevos ligeramente batidos',
 'huevos m',
 'huevos camperos',
 'huevos se',
 'huevos frescos',
 'cuatro huevos',
 'dos huevos',
 '•huevos',
 'ocho huevos',
 'unidad huevos',
 'ud. huevos',
 'huevos talla l',
 'huevos xl',
 'huevos cocidos ',
 'huevos crudos',
 'huevos pequeño-medianos',
 'o huevos',
 '1huevos enteros',
 'huevos l ',
 'huevos grandes tamaño',
 'a huevos batidos',
 'unids huevos',
 'huevos grandes.',
 'huevos duros.',
 'huevos batidos ligeramente.',
 'huevos grandes batidos',
 'huevos cocidos y pelados',
 'huevos de fraile',
 'huevos, ',
 'huevos o la cantidad que deseen',
 'y huevos',
 'huevos cocido',
 'unos huevos batidos',
 'huevos blancos',
 'dos de huevos cocidos.',
 'huevos clase l ',
 'unids. huev

In [9]:
filter_pechiga_de_pollo = recipe_df['pechiga de pollo'] > 86
recipe_df[filter_pechiga_de_pollo]["ingredient"].value_counts()

pollo                1004
pechugas de pollo      87
pollo                  39
pollo.                 30
pechuga de pollo       17
- pollo.                3
pollo)                  3
pollo:                  1
pollo...)               1
- pollo                 1
pollo                   1
Name: ingredient, dtype: int64

In [10]:
filter_lentejas = recipe_df['lentejas'] > 86
recipe_df[filter_lentejas]["ingredient"].value_counts()

lentejas                                   84
lentejas cocidas                           33
lentejas pardinas                          15
lentejas cocinadas                          8
lentejas pardinas cocidas                   8
lentejas                                    6
lentejas secas                              4
lentejas remojadas                          4
lenteja                                     3
lentejas rojas                              2
lentejas cocidas                            2
té                                          2
lentejas amarillas cocidas                  1
lentejas rojas lavados                      1
13gr lentejas                               1
lentejas que se encuentre en la página)     1
lentejas amarillas                          1
a                                           1
lentejas remojadas en agua                  1
lentejas cocidas en su jugo                 1
paté de lentejas                            1
10g lentejas francesas            

In [44]:
filter_pimiento = recipe_df['pimiento'] > 86
recipe_df[filter_pimiento]["ingredient"].value_counts().index.tolist()

['pimienta',
 'pimiento rojo',
 'pimiento verde',
 'pimienta.',
 'pimientos rojos',
 'pimientos verdes',
 'pimiento amarillo',
 'pimiento',
 'pimiento morrón',
 'pimientos del piquillo',
 'pimiento del piquillo',
 'pimiento verde italiano',
 'pimiento rojo asado',
 'pimiento rojo pequeño',
 'pimiento choricero',
 'pimientos morrones',
 'pimiento o pimentón rojo',
 'pimienta ',
 'pimientos',
 'pimientos choriceros',
 'pimientos amarillos',
 'pimiento o pimentón',
 'pimenton',
 'pimiento verde.',
 'pimientos verdes italianos',
 'pimientos rojos asado',
 'pimientos rojos asados',
 '- pimienta.',
 'pimientos asados',
 'pimiento verde pequeño',
 'pimiento morrón rojo',
 '1/pimiento rojo',
 'pimiento rojo grande',
 'pimiento verde ',
 'pimiento ',
 'pimiento morrón verde',
 'pimiento rojo picado',
 'pimiento morrón asado',
 'un pimiento verde',
 'pimiento rojo y verde',
 'pimientos ',
 'pimiento amarillo pequeño',
 '1/pimiento verde',
 'pimientos rojos ',
 'pimiento rojo o verde',
 'pimiento

In [48]:
filter_arroz = recipe_df['arroz'] > 86
recipe_df[filter_arroz]["ingredient"].value_counts().index.tolist()

['arroz',
 'arroz basmati',
 'arroz redondo',
 'arroz blanco',
 'arroz integral',
 'arroz largo',
 'arroz ',
 'arroz bomba',
 'arroz cocido',
 'arroz arborio',
 'arroz blanco cocido',
 'arroz blanco cocinado',
 'harina de arroz',
 'arroz cocido ',
 'vinagre de arroz',
 'arroz salvaje',
 'arroz integral ',
 'leche de arroz',
 'arroz.',
 'arroz carnaroli',
 'río',
 'arroz)',
 'sirope de arroz',
 'arroz inflado de chocolate',
 'arroz blanco ',
 'arroz integral cocido',
 'arroz crudo',
 'arroz jazmín',
 'fideos de arroz',
 'morcilla de arroz',
 'oblea de arroz',
 'arroz arborio o carnaroli',
 'arroz nerone (venere)',
 'arroz integral.',
 'arroz o mirin',
 'arroz pequeñas',
 'arroz basmati cocido',
 'arroz inflado',
 'arroz arboreo',
 'pasta de arroz',
 'arroz hervido',
 'arroz blanco.',
 'arroz cocido sin sal',
 'arroz integral cocinado',
 'arroz arroz cocido',
 'arroz salvaje cocido',
 'arroz tres delicias',
 'arroz tipo integral',
 'un vaso de arroz integral',
 'fideos o arroz',
 'un kil

In [49]:
filter_leche = recipe_df['leche'] > 86
recipe_df[filter_leche]["ingredient"].value_counts().index.tolist()

['leche',
 'leche ',
 'leche condensada',
 'leche entera',
 'leche evaporada',
 'leche o nata',
 'leche desnatada',
 'chocolate con leche',
 'leche vegetal',
 'leche de coco',
 'leche en polvo',
 'leche de soja',
 'leche tibia',
 'chocolate con leche fondant',
 'leche.',
 'leche)',
 'crema de leche',
 'leche condensada ',
 'leche o nata líquida',
 'dulce de leche',
 'leche semidesnatada',
 'leche de almendras',
 '¾ taza de leche ',
 'leche de arroz',
 'leche líquida',
 'leche entera.',
 'leche caliente',
 'leche de avena',
 'leche fría',
 'leche templada',
 'leche fresca',
 'leche o nata ',
 'leche vegetal ',
 'leche descremada',
 '- leche.',
 'crema de leche.',
 'leche o agua',
 'leche o leche evaporada',
 'leche evaporada ',
 'agua o leche',
 '3gr leche',
 '5gr leche',
 '10gr leche',
 'leche espesa',
 'medio litro de leche',
 'leche condensada 1 bote pequeño',
 'lata leche evaporada',
 'leche o nata.',
 'leche hervida',
 'l. leche',
 '1/l. leche',
 'chocolate con leche de cobertura',

In [50]:
filter_mantequilla = recipe_df['mantequilla'] > 86
recipe_df[filter_mantequilla]["ingredient"].value_counts().index.tolist()

['mantequilla',
 'mantequilla sin sal',
 'mantequilla ',
 'mantequilla.',
 'mantequilla derretida',
 'mantequilla o margarina',
 'maní',
 'mantequilla a temperatura ambiente',
 '10gr mantequilla',
 'mantequilla blanda',
 'mantequilla fundida',
 '5g mantequilla',
 'mantequilla con sal',
 'mantequilla reblandecida',
 'mantequilla una nuez',
 'manteca o mantequilla',
 'aceite o mantequilla',
 'mantequilla pomada',
 'mantequilla fría',
 'margarina o mantequilla',
 'mantequilla o aceite',
 'mantequilla clarificada',
 'mantequilla ablandada',
 'mantequilla sin sal a temperatura ambiente',
 '2g mantequilla',
 '3g mantequilla',
 '5gr mantequilla',
 'mantequilla vegetal',
 'matequilla',
 'mantequilla al gusto',
 'mantequilla salada',
 'mantequilla de cacahuete',
 'mantequilla nuez',
 'cucharada mantequilla',
 '5gr. mantequilla',
 'cuchara sopera mantequilla',
 'mantequilla fría sin sal',
 'mantequilla fresca',
 'mantequilla en pomada',
 '4g mantequilla',
 '15gr mantequilla',
 '7gr. mantequilla'

In [51]:
filter_harina = recipe_df['harina'] > 90
recipe_df[filter_harina]["ingredient"].value_counts()

harina       3008
harina        346
harina.        24
hariana         2
harina .        2
- harina        1
y harina        1
harina,         1
- harina        1
Name: ingredient, dtype: int64

In [52]:
filter_sal = recipe_df['sal'] > 90
recipe_df[filter_sal]["ingredient"].value_counts()

sal       12040
sal.        243
sal          49
- sal.       14
- sal         3
•sal          3
•sal.         2
sal,          2
sal,          1
-sal          1
-sal .        1
sal ,         1
-sal.         1
sal)          1
sal .         1
Name: ingredient, dtype: int64

In [53]:
filter_azúcar = recipe_df['azúcar'] > 90
recipe_df[filter_azúcar]["ingredient"].value_counts()

azúcar       3118
azúcar        646
azucar         57
azúcar.        19
azùcar          7
azucar.         3
•azúcar         3
azucar          3
azùcar.         2
azúcar,         2
-azucar.        1
- azucar        1
-azúcar .       1
- azúcar        1
6azúcar         1
Name: ingredient, dtype: int64

In [57]:
filter_aceite = recipe_df['aceite'] > 86
recipe_df[filter_aceite]["ingredient"].value_counts().index.tolist()

['aceite de oliva',
 'aceite',
 'aceite ',
 'aceite vegetal',
 'aceite de oliva virgen extra',
 'aceite de girasol',
 'aceite virgen extra hojiblanca',
 'aceite de oliva virgen',
 'aceite de oliva suave',
 'atún en aceite',
 'aceite oliva virgen extra',
 'aceite de oliva.',
 'aceite oliva',
 'aceite.',
 'aceite de oliva ',
 'anchoas en aceite',
 'atún en conserva en aceite',
 'aceite neutro',
 'cucharadas aceite',
 'aceite de girasol ',
 'tomate seco en aceite',
 'atún en conserva en aceite de oliva',
 'filetes de anchoa en aceite',
 'tomates secos en aceite',
 'aceite de coco',
 'anchoa en aceite',
 'caballa en aceite',
 'aceite virgen extra',
 'bonito en aceite',
 'aceite o mantequilla',
 'aceite suave',
 'aceite y sal',
 'aceite girasol',
 'aceite de sésamo',
 'mantequilla o aceite',
 'aceite el necesario',
 'medio vaso de aceite de oliva',
 'aceite en aerosol',
 'aceite vegetal ',
 'cch. aceite',
 'aceite de oliva virgen extra ',
 'un poco de aceite',
 'cdas aceite',
 'aceite suave

In [58]:
filter_zanahorias = recipe_df['zanahorias'] > 86
recipe_df[filter_zanahorias]["ingredient"].value_counts().index.tolist()

['zanahoria',
 'zanahorias',
 'zanahorias medianas',
 'zanahorias.',
 'zanahorias pequeñas',
 'zanahoria.',
 'zanahorias baby',
 'zanahorias grandes',
 'zanahoria ',
 'zanahorias rallada',
 'zanahorias ralladas',
 'zanahorias picadas',
 'zanahorias ',
 'zanahorias en rodajas',
 'zanahorias cocidas',
 'tres zanahorias',
 'zanahorias peladas y cocidas',
 'a zanahorias',
 'zanahorias picadas en cuadritos',
 'zanahorias medianas cortadas en cuadros.',
 'zanahorias escurtidas picadas',
 'puerros, zanahorias, ajo, perejil...',
 'zanahorias con las hojas',
 'zanahorias limpias y troceadas',
 '20grs. zanahorias',
 'zanahotias',
 'o zanahorias',
 'zanahorias ralladas gruesas',
 'zanahorias partidas en ruedas ',
 'zanahorias cortadas en bastones',
 'zanahorias medianas cortadas en trozos',
 'zanahorias en julianas',
 'zanahorias, cortadas a la juliana',
 'zanahorias frescas',
 'dos zanahorias',
 'fondant en forma de zanahorias',
 'zanhorias',
 'zanahorias cortadas en dos',
 'piña y zanahorias en

In [20]:
filter_queso = recipe_df['queso'] > 86
recipe_df[filter_queso]["ingredient"].value_counts()

queso rallado                             466
queso parmesano                           341
queso blanco                              264
queso mozzarella                          240
queso parmesano rallado                   207
                                         ... 
300g. queso fresco                          1
queso blanco o mozzarella                   1
5gramos queso                               1
lonchas de queso gruyère o de emmental      1
queso cheddar blanco rallado                1
Name: ingredient, Length: 718, dtype: int64

In [21]:
filter_bacon = recipe_df['bacon'] > 86
recipe_df[filter_bacon]["ingredient"].value_counts()

bacon                        122
bacon ahumado                 20
bacon o tocineta              14
tocineta o bacon              10
bacón                          9
                            ... 
panceta ahumada o bacon        1
bacon cortado en trocitos      1
tocino o bacon ahumado         1
lonchas de bacon               1
20g bacon fresco               1
Name: ingredient, Length: 71, dtype: int64

In [22]:
filter_lechuga = recipe_df['lechuga'] > 86
recipe_df[filter_lechuga]["ingredient"].value_counts()

lechuga                                                 325
lechuga romana                                           40
lechugas                                                 37
cogollo de lechuga                                       18
lechuga iceberg                                          15
                                                       ... 
lechugas hojas variadas (cogollos, escarola, etc...)      1
lechuga brotes tiernos variados                           1
lechuga rizada o escarola                                 1
lechuga grande                                            1
lechuga americana                                         1
Name: ingredient, Length: 128, dtype: int64

In [23]:
filter_calabacín = recipe_df['calabacín'] > 80
recipe_df[filter_calabacín]["ingredient"].value_counts()

calabacín                      473
calabacines                     58
calabacínes                     51
calabacín verde                 36
calabacín pequeño               23
                              ... 
calabacín torneado               1
calabacín no muy grande          1
calabacín bola                   1
flor de calabacín                1
calabacín mediano o pequeño      1
Name: ingredient, Length: 71, dtype: int64

In [24]:
filter_patatas = recipe_df['patatas'] > 86
recipe_df[filter_patatas]["ingredient"].value_counts()

patatas                                677
patata                                 354
patatas medianas                       137
patatas pequeñas                        52
patatas grandes                         52
                                      ... 
patatas medianas                         1
patatas medianas picadas en juliana      1
patatas en cuadros                       1
patatas asadas                           1
1patatas pequeñas-medianas               1
Name: ingredient, Length: 150, dtype: int64

In [25]:
filter_pollo = recipe_df['pollo'] > 86
recipe_df[filter_pollo]["ingredient"].value_counts()

pollo                                 1004
pechugas de pollo                       87
caldo de pollo                          82
pollo en polvo                          64
pollo troceado                          44
                                      ... 
pollo granulado                          1
pavo, pollo o carne                      1
pechugas de pollo en filetes finos       1
fingers de pollo                         1
pollo grande partido en piezas           1
Name: ingredient, Length: 381, dtype: int64

In [26]:
filter_atun = recipe_df['atún'] > 86
recipe_df[filter_atun]["ingredient"].value_counts()

atún                         184
atún en aceite                61
atún en conserva              50
atún en agua                  32
atún al natural               13
                            ... 
atún o bonito                  1
atún en rodajas gruesas        1
atún en agua.                  1
bonito en conserva o atún      1
lata atún                      1
Name: ingredient, Length: 76, dtype: int64

In [27]:
filter_garbanzos = recipe_df['garbanzos'] > 86
recipe_df[filter_garbanzos]["ingredient"].value_counts()

garbanzos                       179
garbanzos cocidos               116
harina de garbanzos              12
garbanzos cocidos                11
garbanzos                        10
                               ... 
bote o garbanzos cocidos          1
25gr garbanzos                    1
un bote de garbanzos cocidos      1
garbanzos castellanos             1
un kilo de garbanzos              1
Name: ingredient, Length: 62, dtype: int64

In [28]:
filter_judías_verdes = recipe_df['judías verdes'] > 86
recipe_df[filter_judías_verdes]["ingredient"].value_counts()

judías verdes                                   156
judía verde                                      62
judías verdes finas                               8
judías                                            7
judías verdes redondas                            6
judías verdes                                     5
judías verdes cocidas                             3
judías verdes redonda                             3
judías verdes planas cocidas                      3
judías verdes.                                    2
habichuelas o judías verdes                       2
judías verdes planas                              2
verde                                             2
verdes                                            2
judías verdes congeladas                          2
un puñado de judías verdes planas                 1
habichuelines o judías verdes                     1
judías verdes congeladas, troceadas               1
judías verdes en rodajas                          1
10judías ver

In [29]:
filter_jamón = recipe_df['jamón'] > 86
recipe_df[filter_jamón]["ingredient"].value_counts()

jamón serrano          197
jamón                  123
jamón cocido           118
jamón york              40
jamón dulce             24
                      ... 
jamón chorizo            1
jamón cocido molido      1
jamón prosciutto         1
jamón en cubos           1
5gr jamón                1
Name: ingredient, Length: 233, dtype: int64

In [30]:
filter_pan = recipe_df['pan'] > 86
recipe_df[filter_pan]["ingredient"].value_counts()

pan rallado                  514
pan                          269
panadería                     48
pan de molde                  46
pan molido                    45
                            ... 
panificar sino dietética)      1
panceta ibérica curada         1
pan especial                   1
panes                          1
pan de payés o de hogaza       1
Name: ingredient, Length: 400, dtype: int64

In [31]:
filter_pasta = recipe_df['pasta'] > 86
recipe_df[filter_pasta]["ingredient"].value_counts()

pasta                                   59
pasta                                   53
pasta brisa                             52
pasta de ajo.                           44
pasta corta                             43
                                        ..
pza paquete pasta caracol chico          1
tondarelli u otra pasta fresca larga     1
ají en pasta                             1
10gr. harina en pasta                    1
pasta corta cocinada                     1
Name: ingredient, Length: 217, dtype: int64